# An introduction to Asyncio
In this tutorial we will learn the basics of asyncio.     
Hopefully, we will be able to build a distributed task executor.

## What is Asyncio?
According to the docs:

> asyncio is a library to write concurrent code using the async/await syntax.

> asyncio is often a perfect fit for IO-bound and high-level structured 
 network code.

## Outline

- Why Asyncio?
- A detour in the Coroutine world
- Asyncio Coroutine, Future, Task
- The select module
- Transport and Protocols
- Putting it all together

# Why do we need Asyncio?
- I/O is really slow
- SSD latency time is ~1ms, 1Ghz Core >> 1Mln CPU wasted cycles at second 

## Asyncio vs Threads
1. Asyncio is typically single-threaded
2. You don't have to worry about synchronisation (Locks, Semaphores, Queues)
3. Application controls the context switching
    -  In a multi-threaded program the context switching is controlled by the OS
4. Asyncio for I/O-bound tasks, Threads for CPU-bound tasks

# How can we do concurrency without threads?
> You don't need threads to have concurrency   

> You don't need multi-core to have concurrency

> You don't need a fast CPU to have concurrency

## Time Slicing or Sharing

You do a little bit of work for one task, you switch and
 you do a little piece of work for a different task (time-slicing).    
The OS takes the control of the CPU and gives a different task the access to it.

You do that for a lot of times every seconds and it looks like the tasks are all running concurrently.

In [74]:
def pippo():
    yield "Hi, who are you?"
    yield "Nice to meet you."
    yield "I am Pippo"

def pluto():
    yield "Hi, I am Pluto"
    yield "What's your name?"

loop = [pippo(), pluto()]
while loop:
    el = loop.pop(0)
    try:
        print(next(el))
        loop.append(el)
    except StopIteration:
        pass
 

Hi, who are you?
Hi, I am Pluto
Nice to meet you.
What's your name?
I am Pippo


In [75]:
# Generators are bi-directional (Coroutines)

loop = [pippo(), pluto()]
while loop:
    el = loop.pop(0)
    try:
        print(el.send(None))
        loop.append(el)
    except StopIteration:
        pass

Hi, who are you?
Hi, I am Pluto
Nice to meet you.
What's your name?
I am Pippo


# What have we got here?
- Two generators running sort of concurrently
- Application defined context switch points 
- A loop to dispatch them

# Coroutines Detour
- In Python 2.5 generators picked up some new features to allow coroutines
- Essentially a new `send` method

## Generators 
A generator is a function that produces a sequence of results instead of a
 single value
 

In [76]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

for i in countdown(5):
    print(i)

5
4
3
2
1


In [77]:
x = countdown(3)
print(x)
print(next(x))
print(next(x))
print(next(x))
print(next(x))

<generator object countdown at 0x10e3b5750>
3
2
1


StopIteration: 

## Generators as pipeline
We know how to use generators to build pipelines.   
We just need to pass each generator to the next generator. 

In [82]:
def odd(prev):
    for el in prev:
        if el % 2 == 1:
            yield el
            
def mul(prev, factor):
    for el in prev:
        yield el * factor
        
a = odd(range(10))
b = mul(a, 2)

for line in b:
    print(line)

2
6
10
14
18


## Yield as an Expression
In Python 2.5, a slight modification to the yield statement was introduced
 (PEP-342). You can now use the yield statement as an expression.

In [83]:
def odd(nextcoro):
    while True:
        num = (yield)
        if num % 2 == 1:
            nextcoro.send(num)

def mul(factor, nextcoro):
    while True:
        num = (yield)
        nextcoro.send(num * factor)

def printer():
    while True:
        num = (yield)
        print(num)

p = printer()
m = mul(2, p)
o = odd(m)
# coroutines needs to be "primed" first calling next() or send(None)
# just write your own decorator not to forget to prime your @coroutine
p.send(None)
m.send(None)
o.send(None)

for i in range(10):
    o.send(i)

2
6
10
14
18


- You can also `close()` which can be caught with `GeneratorExit` exception
- You can also `throw()` exceptions inside the coroutine
- It is easier to broadcast results to several coroutines if not even
 more complex branching when designing pipelines 

## Generator vs Coroutine 
- Generators produce data for iteration
- Coroutines are consumers of data
- ... But it's python, which means you can have a generator consuming data
 as a coroutine.   

In [84]:
def countdown(n):
    while n >= 0:
        new_value = (yield n)
        if new_value is not None:
            n = new_value
        else:
            n -= 1

c = countdown(5)
for n in c:
    print(n)
    if n == 5:
        c.send(3)

5
2
1
0


In asyncio the generators are called coroutines   
The loop that executes them is the event loop

# Future
A future is an indirect reference to a forthcoming result   
You can ask the future to `callback` when ready

In [85]:
import asyncio

def pippo(when_pluto_is_done):
    print("Hi, who are you?")

    def nice_to_meet_you(ret):
        print("Nice to meet you.")
    
    when_pluto_is_done.add_done_callback(nice_to_meet_you)


def pluto(when_pippo_is_done):
    print("Hi, I am Pluto")
    
    when_pippo_is_done.set_result(None)


loop = asyncio.get_event_loop()
future = loop.create_future()

pippo(future)
pluto(future)

await future

Hi, who are you?
Hi, I am Pluto
Nice to meet you.


# Callbacks
- Callbacks aren't the nicest way to do things
- Wouldn't it be great to write the code inline like before?

# Asyncio Coroutines 
Coroutines are declared with async/await syntax is the preferred 
way of writing asyncio applications  

The await is a fancy way of doing the yield.  

Essentially, the event loop is going to handle callbacks for us    
and allows us to write our code in a nice clean logical way    

In [87]:
# Three way to run a coroutine:
# - The asyncio.run() function to run the top-level entry point "main()" function
# - Awaiting on a coroutine.

import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

await main()

started at 23:59:14
hello
world
finished at 23:59:17


## Tasks
- A task executes a coroutine in an event loop 
- At each step the coroutine either
    - awaits a future
    - awaits another coroutine
    - returns a result

In [88]:
# The asyncio.create_task() function to run coroutines 
# concurrently as asyncio Tasks.
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")
    
await main()

started at 23:59:43
hello
world
finished at 23:59:45


## Select Module
- select is an OS function to wait for I/O
- it tells you which I/O channels are ready
- I/O channels can be files, sockets or pipes
- It can wait a fixed length of time or indefinitely

The event loop use this select method to figure out when I/O is ready and do
 something with it.

# Asyncio Transport
- Transports are communication channels
- Responsible for performing I/O 
- Several types: TCP, UDP, SSL, Pipes

# Streaming Transport
For example TCP    
The API includes methods such as:
- close, write, pause/resume reading

Note: no read method
- instead you get a callback (you do not want to block your code execution)

- You don't create transports directly
- Instead the event loop supplies methods 
- For example:
    - `create_connection`
    - `create_server`
- Each takes a protocol factory as its first argument

# Asyncio Protocols
What the Transport only does it sends data from a point to another.    
To interpret the data you need a protocol.

- Asyncio protocols process received data and ask the transport to send data
- Your application will create subclasses of a protocol to define how to behave in case of received data.

# Putting all together

## References
- [Coroutines](http://dabeaz.com/coroutines/Coroutines.pdf)
- [Youtube Video of Coroutines](https://www.youtube.com/watch?v=Z_OAlIhXziw)
- [PEP-342](https://www.python.org/dev/peps/pep-0342/)